# OCCRP’s COVID-19 Procurement Explorer
*OCCRP and media partners collected data on COVID-19 related spending from across Europe from February to October this year.
The story analyzing the data can be found [here](https://www.occrp.org/en/coronavirus/europes-covid-19-spending-spree-unmasked).*

*We have also decided to share the raw data with the public.*


In this notebook, we will clean the data for analysis and explain some of the features and caveats of the data along the way. We have collected data into **two data sets**: tenders and contracts, and unit prices.


## Source data sets

### 1. TENDERS

This dataset contains information on contracts and tenders related to COVID-19. This information can be found [here](https://docs.google.com/spreadsheets/d/1VXURZlKH-_GeNvPrytgJOeTUH3hXf0r_veIXWJp1K20/edit?usp=sharing).
A quick note here on the difference between contracts and tenders. Where as contracts typically cover a simple, one-off purchase from a single company, tenders are often divided into multiple parts, or “lots”, with more than one company acting as supplier.
This means there is sometimes duplication in this data. The data is structured around companies (i.e. one row = one winning company). If a tender has multiple winning companies, that means it spans over multiple rows.

We have also made an effort to categorize each tender or contract (visible in the `product` column), in order to filter out those not related to COVID-19, and to make it easier to compare different deals with each other. This process was done with a mix of manual edits and automation, meaning that there may be mistakes.


### 2. UNIT PRICES

This dataset contains information on the prices paid per unit for certain COVID-19 purchases. The information can also be found in the [Unit Prices Sheet](https://docs.google.com/spreadsheets/d/10VL5FpviSXctagcoQM_pr0xP4Lsmzzc3-i7mEyCE2kw/edit?usp=sharing). 
This data comes from multiple sources. In Ukraine, for example, data came from the [Prozorro](https://prozorro.gov.ua/en/tender/search/) procurement platform. In Portugal, data was obtained from the government and filtered and categorized by our media partner [Publico](https://www.publico.pt). Czech data was filtered by [hlidacstatu.cz](https://www.hlidacstatu.cz/).
Most of the time there is a reference to a source and an ID corresponding to the relevant tender or contract.


# Data frames
In this notebook, we are pulling in the raw data, cleaning it and creating three data frames for you to explore:


1. **TENDERS** : Contains information on tenders and contracts for COVID-19 related services and equipment on the tender/contract level. Contains information on price, date, descriptions, and buyers (NOT companies).

2. **COMPANIES** :  Contains information on the companies that have sold COVID-19 related services and equipment. Information includes company details, price, date, descriptions, and buyers.

3. **UNIT_PRICES** : Contains information on the unit prices of a particular COVID-19 related equipment.


## A note on working with tender data

Tender data is notoriously difficult to work with. The most common caveats are:

* The data is **incomplete** in many ways. Most of the countries don't publish all of their expenses. In some countries (e.g. Portugal, Spain, Czechia, Russia, UK) nearly all contracts are openly published. In others (e.g. Germany, France, the Netherlands) only the largest, so called EU-level tenders are published. The largest tenders account for only about ~1/3 of total expenditures. Even if a tender is published, some of the information is often missing. It's not unusual to see for example the price either missing, or even set to zero or one euro.

* The **company names are not standardized**. Governments can fill in company names in many ways and so make it very difficult to calculate how much a particular company is making from tenders in general.


Part of this repository is a collection of cleaning scripts called `covidtenders`. `pandas` is used for the rest.


In [19]:
import pandas as pd

import covidtenders
from importlib import reload
reload(covidtenders)

pd.set_option("max_colwidth", 500)
pd.set_option("max_rows", 100)
pd.set_option("max_columns", 50)
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('mode.chained_assignment', None)

import warnings
warnings.filterwarnings("ignore")

In [20]:
# load the whole TENDERS data set
alldata = pd.read_csv("source_tables/alldata.csv")

# Cleaning the TENDERS for TENDERS (see `covidtenders.py` for details on the steps)
#TENDERS = covidtenders.filter_just_tenders(alldata)

# Cleaning the for the COMPANIES (see `covidtenders.py` for details on the steps)
COMPANIES = covidtenders.filter_just_companies(alldata,
                                               missing_prices_filled_in_dataset = "source_tables/companies_prices_missing_filled_in.csv",
                                               companies_deduped = "source_tables/companies_deduped.csv")

# Loading and cleaning UNIT PRICES (see `covidtenders.py` for details on the steps)
#UNIT_PRICES = pd.read_csv("source_tables/unit_prices.csv")
#UNIT_PRICES = covidtenders.clean_unitprices(UNIT_PRICES)

TENDERS.to_csv("exports/occrp-covid19-tenders.csv", index=False)
COMPANIES.to_csv("exports/companies_final.csv", index=False)
UNIT_PRICES.to_csv("exports/unit-prices-clean.csv", index=False)



+++ Cleaning TENDER data for COMPANY details +++
ORIGINAL size: 56461 rows
... after cleaning: 41341 rows
-----------
Step 1: contracts + single win tenders from + single win lots from + unique tenders = 39255 winning bids
              date      product            type  \
ID                                                
AT0004  2020-04-06  Ventilators  awarded tender   
AT0005  2020-03-16  Ventilators  awarded tender   
AT0006  2020-03-16  Ventilators  awarded tender   
AT0007  2020-03-16  Ventilators  awarded tender   
AT0008  2020-03-13  Ventilators  awarded tender   

                                  title  lot buyer_country  \
ID                                                           
AT0004  COVID-Beatmungsgeräte & Zubehör  NaN            AT   
AT0005                  Beatmungsgeräte  NaN            AT   
AT0006                  Beatmungsgeräte  NaN            AT   
AT0007  COVID-Beatmungsgeräte & Zubehör  NaN            AT   
AT0008  Covid-Beatmungsgeräte & Zubehör  NaN  

## TENDERS

**Biggest spenders** : 
In the UK, tender data is open by default. Germany only publishes in TED. 

In [26]:
TENDERS.groupby("buyer_country")["price_EUR"].sum().reset_index().nlargest(15,"price_EUR").style.bar()
# (EC = European Commission)

,buyer_country,price_EUR
37,UK,10030816670.970715
10,EC,3465894674.290000
8,DE,2081221755.359999
20,IT,904554970.900000
12,ES,800985771.000000
1,AT,706154997.001950
32,RU,676598513.852159
30,PT,463817905.663480
14,FR,286789780.890000
11,EE,245059046.000000


**Biggest publishers of tenders** : Portugal is our best data set. Portugal publishes all contracts on a designated website and the data collection was regurarly updated, cleaned and categorized by our partner [Publico](https://www.publico.pt/). Portugeese data have therefore the potential to be used as a base line for how the pandemic spending looked like. Lithuania is another case where the data was regurarly collected, cleaned and published directly by the government publication office (but we did not succeed in categorizing them all).

In [27]:
TENDERS.groupby("buyer_country")["product"].count().reset_index().nlargest(15,"product").style.bar()

,buyer_country,product
30,PT,15217
32,RU,12364
36,UA,2229
31,RO,1615
24,MD,1208
7,CZ,979
37,UK,616
21,LT,606
1,AT,536
12,ES,329


## COMPANIES
The **50 largest suppliers**

In [28]:
COMPANIES.groupby("supplier_clean")["price_contract_EUR"].sum().reset_index().nlargest(50, "price_contract_EUR").style.bar()

,supplier_clean,price_contract_EUR
4672,gyz,1493130848.360000
11349,unispace global,526550156.700000
10562,sws schüler,410676100.000000
8099,optigene ltd,364721313.120000
11348,uniserve ltd,327095450.000000
8215,p14 medical,312441573.840000
1088,ayanda capital ltd,285325000.000000
1667,byd auto industry company ltd,271629440.000000
109,abbott,254398787.190000
11803,vyaire,246514957.440000


**!!!** We also encountered cases of some tenders **duplicated in the source data** (e.g. the TED database), see an example here [[1]](https://ted.europa.eu/udl?uri=TED:NOTICE:240431-2020:TEXT:EN:HTML) [[2]](https://ted.europa.eu/udl?uri=TED:NOTICE:288971-2020:TEXT:EN:HTML). We **did not clean** these. See example below.

In [29]:
COMPANIES[COMPANIES["supplier"] == "Müller und Meirer Lederwaren"]

# uncomment below to see more possible duplicates
# companies[(companies.duplicated(["title","price_contract_EUR", "supplier"]))].sort_values("price_contract_EUR", ascending=False)

,date,product,type,title,lot,buyer_country,supplier,price_contract_EUR,price_EUR,price_lot_EUR,supplier_id,supplier_country,supplier_city,supplier_street,supplier_postcode,procedure,bids,contract_number,TED_id,price,price_currency,price_lot,source,supplier_clean,diff
1779,2020-04-23,PPE (protective equipment),awarded tender,"20-00155_01, Beschaffung von Einweghandschuhen aufgrund der Corona-Epidemie",1,DE,Müller und Meirer Lederwaren,"15,750,000.00","15,750,000.00","15,750,000.00",NaN,DE,Kim,Am Markt 8,55606,direct award,1.00,20-00155_01,2020/S 100-240431,15750000.0,EUR,15750000.0,http://ted.europa.eu/udl?uri=TED:NOTICE:240431-2020:TEXT:DE:HTML,müller und meirer lederwaren,0.35
1780,2020-04-23,PPE (protective equipment),awarded tender,"20-00155_01, Beschaffung von Einweghandschuhen aufgrund der Corona-Epidemie",1,DE,Müller und Meirer Lederwaren,"15,750,000.00","15,750,000.00","15,750,000.00",NaN,DE,Kirn,Am Markt 8,55606,direct award,1.00,20-00155_01,2020/S 119-288971,15750000.0,EUR,15750000.0,http://ted.europa.eu/udl?uri=TED:NOTICE:288971-2020:TEXT:DE:HTML,müller und meirer lederwaren,0.35


### UNIT_PRICES
Most frequent countries of origin

In [30]:
UNIT_PRICES["country"].value_counts().reset_index().style.bar()

,index,country
0,Ukraine,3731
1,Portugal,1523
2,Moldova,1265
3,Lithuania,948
4,Czechia,714
5,Sweden,537
6,Austria,294
7,Spain,169
8,Greece,155
9,Poland,143


The most *often occuring* category are **medical gloves** (these might not be comparable because of different types of gloves)

In [31]:
UNIT_PRICES["Product category"].value_counts().reset_index().head(10).style.bar()

,index,Product category
0,MEDICAL GLOVES,2046
1,FFP2,1278
2,OTHER MEDICAL EQUIPMENT,438
3,MASK (UNSPECIFIED),364
4,OTHER: NON MEDICAL,341
5,DISPOSABLE COVERALL,333
6,MEDICINES AND OTHER PILLS,291
7,VENTILATORS,249
8,DISINFECTANTS,246
9,SURGICAL MASK,229


The items bought in *largest amounts* were **surgical masks**. These most probably refer to the disposable 3-ply masks.

In [32]:
UNIT_PRICES.groupby("Product category")["amount"].sum().reset_index().nlargest(10,"amount").style.bar()

,Product category,amount
49,SURGICAL MASK,671134258.000000
15,FFP2,460785542.625966
34,MEDICAL GLOVES,363459170.632414
27,KN95,92358193.000000
36,N95,88529585.000000
33,MASK TYPE II,55666542.000000
38,OTHER MEDICAL EQUIPMENT,36386202.500000
16,FFP3,22254634.000000
31,MASK (UNSPECIFIED),16394699.398197
10,DISPOSABLE COVERALL,14841353.100000


However, the *most money* was spent on **FFP2 and equivalent - (K)N95 respirators**. As these is a standardized item, the prices are comparable cross-border

In [33]:
UNIT_PRICES.groupby("Product category")["spent"].sum().reset_index().nlargest(10,"spent").style.bar()

,Product category,spent
15,FFP2,1571087041.695052
49,SURGICAL MASK,267498957.265798
27,KN95,160183274.485131
36,N95,139517236.190000
10,DISPOSABLE COVERALL,129733130.224320
16,FFP3,108829255.889650
53,VENTILATOR,88195305.130943
6,COVID19 TEST KIT,59172731.068550
5,COVID19 TEST,58803113.552679
38,OTHER MEDICAL EQUIPMENT,55355381.392337
